In [103]:
%reset
from pyparsing import Word, hexnums, WordEnd, Optional, alphas, alphanums
from collections import defaultdict

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [109]:
address_format = Word(hexnums, exact=8) + WordEnd() # use WordEnd to avoid parsing leading a-f of non-hex numbers as a hex
byte_format = Word(hexnums, exact=2) + WordEnd()
instrn_line_format = ".text:" + address_format + (byte_format*(1,))("bytes") + Word(alphas,alphanums)("instruction")
byte_line_format = address_format + (byte_format*(1,))("bytes")

In [117]:
# Globals
SAMPLES_BASE_DIR = 'samples/'
TEST_FILES = ['0A32eTdBKayjCWhZqDOQ', '0ACDbR5M3ZhBJajygTuf']
INSTRN_BIGRAM_THRESHOLD = 20
BYTE_BIGRAM_THRESHOLD = 100

In [120]:
for filename in TEST_FILES:
    instrn_unigram = defaultdict(int)
    instrn_bigram = defaultdict(int)
    byte_unigram = defaultdict(int)
    byte_bigram = defaultdict(int)
    with open(SAMPLES_BASE_DIR + filename + ".asm", 'r', encoding='Latin-1') as file:
        prev, now = 0, 0
        for line in file:
            # Filtering lines
            if not line.startswith('.text'):
                continue
            if ' db ' in line or ' dd ' in line or ' dw ' in line or 'align ' in line:
                continue
                
            try:
                result = instrn_line_format.parseString(line)
            except:
                continue
                
            prev = now
            now = result.instruction
            instrn_bigram[(prev, now)] += 1
            instrn_unigram[now] += 1
#                 if result.instruction == 'CC':
#                     print(line)
    instrn_bigram = defaultdict(int, {k:v for k,v in instrn_bigram.items() if v > INSTRN_BIGRAM_THRESHOLD and k[0] != 0})
    print(instrn_unigram)
    print(sum(instrn_unigram.values()))
    print("==========================================================================================")
    print(instrn_bigram)
    print("==========================================================================================")
    with open(SAMPLES_BASE_DIR + filename + ".bytes", 'r', encoding='Latin-1') as file:
        prev, now = 0, 0
        for line in file:
            try:
                result = byte_line_format.parseString(line)
            except:
                continue
            
            byte_list = list(result.bytes)
            for byte in byte_list:
                prev = now
                now = byte
                byte_bigram[(prev, now)] += 1
                byte_unigram[now] += 1

    byte_bigram = defaultdict(int, {k:v for k,v in byte_bigram.items() if v > BYTE_BIGRAM_THRESHOLD and k[0] != 0})
    print(byte_unigram)
    print(sum(byte_unigram.values()))
    print("==========================================================================================")
    print(byte_bigram)
    print("==========================================================================================")
    

defaultdict(<class 'int'>, {'push': 971, 'lea': 367, 'mov': 1923, 'call': 411, 'pop': 471, 'retn': 299, 'jmp': 344, 'test': 498, 'jz': 413, 'add': 194, 'inc': 154, 'jnz': 318, 'sub': 283, 'xor': 412, 'and': 232, 'mul': 5, 'shr': 41, 'imul': 10, 'div': 5, 'cmp': 483, 'jb': 75, 'jbe': 15, 'or': 180, 'ja': 10, 'jnb': 41, 'sbb': 9, 'neg': 12, 'nop': 1, 'leave': 25, 'rep': 9, 'setnz': 2, 'dec': 75, 'jge': 28, 'jg': 16, 'jl': 51, 'jle': 9, 'shl': 84, 'sar': 32, 'not': 25, 'setz': 4, 'setnle': 155, 'movzx': 328, 'cdq': 29, 'jns': 40, 'stosd': 27, 'movsd': 21, 'stmxcsr': 4, 'ldmxcsr': 2, 'fstsw': 3, 'movsx': 3, 'fnstsw': 1, 'fnclex': 1, 'fstcw': 1, 'fldcw': 1, 'fld': 3, 'fistp': 1, 'wait': 5, 'fstp': 4, 'fldz': 1, 'fld1': 1, 'fdivrp': 1, 'fldpi': 1})
9170
defaultdict(<class 'int'>, {('push', 'lea'): 40, ('lea', 'push'): 39, ('push', 'mov'): 275, ('mov', 'call'): 111, ('call', 'mov'): 120, ('mov', 'mov'): 533, ('mov', 'pop'): 97, ('pop', 'retn'): 159, ('retn', 'mov'): 184, ('mov', 'jmp'): 107, 

defaultdict(<class 'int'>, {'push': 25, 'mov': 818, 'add': 9, 'xor': 105, 'cmp': 355, 'jnz': 252, 'or': 51, 'jz': 264, 'lea': 70, 'test': 61, 'dec': 2, 'and': 48, 'jmp': 85, 'pop': 14, 'leave': 2, 'retn': 4, 'call': 9, 'sub': 1, 'shr': 1, 'repne': 1, 'inc': 1, 'lodsw': 1, 'stosb': 1})
2180
defaultdict(<class 'int'>, {('xor', 'mov'): 54, ('mov', 'mov'): 426, ('mov', 'cmp'): 176, ('cmp', 'jnz'): 175, ('jnz', 'mov'): 126, ('mov', 'or'): 28, ('or', 'jz'): 29, ('jz', 'lea'): 21, ('lea', 'mov'): 35, ('cmp', 'jz'): 180, ('jz', 'mov'): 138, ('mov', 'lea'): 24, ('test', 'jz'): 28, ('mov', 'xor'): 55, ('mov', 'test'): 33, ('or', 'jnz'): 22, ('jnz', 'cmp'): 71, ('jz', 'cmp'): 62, ('test', 'jnz'): 33, ('and', 'jz'): 27, ('xor', 'cmp'): 26, ('and', 'jnz'): 21, ('jmp', 'jmp'): 83})
defaultdict(<class 'int'>, {'00': 138018, '80': 6006, '40': 6975, '28': 6620, '1C': 27, '02': 7938, '42': 7157, 'C4': 6493, '20': 7839, '04': 6984, '09': 78, '2A': 15197, '8E': 6962, '10': 117843, '41': 78, '0A': 7909, '2